# Semantic Text Similarity

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np

In [7]:
from typing import Tuple, List, Optional

In [8]:
from gensim.models import Word2Vec


In [10]:
from google.colab import drive
drive.mount('/content/drive')
wv_model = Word2Vec.load('/content/drive/MyDrive/word2vec_model_part_3.model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.es.gensim.bin'
    wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')

In [12]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True
MAX_LEN: int = 96

In [13]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("PlanTL-GOB-ES/sts-es")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/77 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/155 [00:00<?, ? examples/s]

In [14]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [15]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [ ]:
print("Max Len:", max([len(s) for s in sentences_1_preproc]), max([len(s) for s in sentences_2_preproc]))
print(list(diccionario.doc2idx(sentences_1_preproc[0])))

Max Len: 87 86
[11, 4, 12, 3, 9, 1, 2, 10, 7, 5, 8, 14, 0, 6, 13]


In [16]:
def map_word_embeddings(
        sentence: str,
        sequence_len: int = MAX_LEN,
        fixed_dictionary: Optional[Dictionary] = None
) -> np.ndarray:
    """
    Map to word-embedding indices
    :param sentence:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    sentence_preproc = simple_preprocess(sentence)[:sequence_len]
    _vectors = np.zeros(sequence_len, dtype=np.int32)
    index = 0
    for word in sentence_preproc:
        if fixed_dictionary is not None:
            if word in fixed_dictionary.token2id:
                # Sumo 1 porque el valor 0 está reservado a padding
                _vectors[index] = fixed_dictionary.token2id[word] + 1
                index += 1
        else:
            if word in wv_model.key_to_index:
                _vectors[index] = wv_model.key_to_index[word] + 1
                index += 1
    return _vectors


def map_pairs(
    sentence_pairs: List[Tuple[str, str, float]],
    sequence_len: int = MAX_LEN,
    fixed_dictionary: Optional[Dictionary] = None
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = map_word_embeddings(sentence_1, sequence_len, fixed_dictionary)
        vector2 = map_word_embeddings(sentence_2, sequence_len, fixed_dictionary)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [17]:
mapped_train = map_pairs(input_pairs, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_val = map_pairs(input_pairs_val, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_test = map_pairs(input_pairs_test, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)

In [ ]:
mapped_train[0]

((array([12,  5, 13,  4, 10,  2,  3, 11,  8,  6,  9, 15,  1,  7, 14,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32),
  array([   5,    4,   10,    2,   21,  958,  381,    5,   14,   21, 1279,
         1158,   62, 1915, 3328,    5, 1014,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,  

In [18]:
# Define model 1
import tensorflow as tf
def model_1(
    input_length: int = MAX_LEN,
    dictionary_size: int = len(diccionario)+1,
    embedding_size: int = 16,
    pretrained_weights: Optional[np.ndarray] = None,
    learning_rate: float = 1e-3,
    trainable: bool = False,
    use_cosine: bool = False,
) -> tf.keras.Model:
    # Input layers
    input_1 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)

    # Embedding layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True
        )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Apply embedding to input sequences
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)

    # Global average pooling
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    pooled_1 = tf.keras.layers.GlobalAveragePooling1D()(embedded_1, mask=_input_mask_1)
    pooled_2 = tf.keras.layers.GlobalAveragePooling1D()(embedded_2, mask=_input_mask_2)

    # Compute similarity/distance
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([pooled_1, pooled_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized

        output = tf.keras.layers.Lambda(normalized_product)([pooled_1, pooled_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)

        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

    # Define the model
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [19]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 128

In [20]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.row_stack(_x_1), np.row_stack(_x_2)), np.array(_y)

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [21]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [22]:
_pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        _pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model.wv:
             _pretrained_weights[_id + 1] = wv_model.wv[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        _pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        _pretrained_weights[1:, :] = wv_model.vectors

In [ ]:
_pretrained_weights[:5,:]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

In [23]:
# Build and compile the model
model = model_1(pretrained_weights=_pretrained_weights, trainable=False, use_cosine=False, ) #PRE_TRAINED_WEIGHTS = None PER ALEATORI
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 96)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 96, 100)              1145700   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 96)                   0         ['input_1[0][0]']         

In [24]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/128
21/21 [==============================] - 1s 13ms/step - loss: 1.1464 - val_loss: 1.3414
Epoch 2/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0776 - val_loss: 1.2864
Epoch 3/128
21/21 [==============================] - 0s 7ms/step - loss: 1.0562 - val_loss: 1.2706
Epoch 4/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0494 - val_loss: 1.2669
Epoch 5/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0531 - val_loss: 1.2654
Epoch 6/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0445 - val_loss: 1.2633
Epoch 7/128
21/21 [==============================] - 0s 8ms/step - loss: 1.0490 - val_loss: 1.2604
Epoch 8/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0512 - val_loss: 1.2572
Epoch 9/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0488 - val_loss: 1.2541
Epoch 10/128
21/21 [==============================] - 0s 6ms/step - loss: 1.0397 - val_loss: 1.2508
Epoch 11

In [25]:
from scipy.stats import pearsonr

x_test, y_test = pair_list_to_x_y(mapped_test)
def compute_pearson(x_, y_, model):
    # Get predictions for the model
    y_pred = model.predict(x_)
    # Compute pearson correlation
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

In [26]:
# Print results
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train, model)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val, model)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test, model)}")

42/42 [==============================] - 0s 6ms/step
Correlación de Pearson (train): 0.5534437193830095
3/3 [==============================] - 0s 5ms/step
Correlación de Pearson (validation): 0.5650258581902237
5/5 [==============================] - 0s 5ms/step
Correlación de Pearson (test): 0.45910266673651123
